In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from glob import glob as g
import cv2
from tqdm.notebook import tqdm
from shutil import copy, move

In [ ]:
import os
import cv2
from shutil import copy
from tqdm import tqdm

def create_dataset(images_dir, labels_dir, label_text_path, images_address):
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)
    annots = open(label_text_path)
    lines = annots.readlines()
    names = [x for x in lines if 'jpg' in x]
    indices = [lines.index(x) for x in names]

    for n in tqdm(range(len(names))):
        i = indices[n]
        name = lines[i].rstrip()
        old_img_path = os.path.join(images_address, name)
        name = name.split('/')[-1]
        label_path = os.path.join(labels_dir, name.split('.')[0] + '.txt')
        img_path = os.path.join(images_dir, name)

        num_objs = int(lines[i + 1].rstrip())
        bboxs = lines[i + 2: i + 2 + num_objs]
        bboxs = list(map(lambda x: x.rstrip(), bboxs))
        bboxs = list(map(lambda x: x.split()[:4], bboxs))

        img = cv2.imread(old_img_path)
        img_h, img_w, _ = img.shape

        with open(label_path, 'w') as f:
            count = 0
            for bbx in bboxs:
                x1 = int(bbx[0])
                y1 = int(bbx[1])
                w = int(bbx[2])
                h = int(bbx[3])

                x = (x1 + w // 2) / img_w
                y = (y1 + h // 2) / img_h
                w = w / img_w
                h = h / img_h

                if w * h * 100 > 2:
                    yolo_line = f'{0} {x} {y} {w} {h}\n'
                    f.write(yolo_line)
                    count += 1

        if count > 0:
            copy(old_img_path, img_path)
        else:
            os.remove(label_path)


train_images_dir = 'newDataset/images/train'
train_labels_dir = 'newDataset/labels/train'
train_label_text_path = 'wider_face_split/wider_face_train_bbx_gt.txt'
train_images_address = 'WIDER_train/images'

val_images_dir = 'newDataset/images/val'
val_labels_dir = 'newDataset/labels/val'
val_label_text_path = 'wider_face_split/wider_face_val_bbx_gt.txt'
val_images_address = 'WIDER_val/images'

create_dataset(train_images_dir, train_labels_dir, train_label_text_path, train_images_address)
create_dataset(val_images_dir, val_labels_dir, val_label_text_path, val_images_address)

In [ ]:
import cv2
import glob
from tqdm import tqdm
import os

def resize_image(input_path, output_path, target_width=1280):
    """
    Изменение размера изображения с сохранением пропорций и запись в новый файл.

    :param input_path: str, путь к исходному изображению
    :param output_path: str, путь для сохранения измененного изображения
    :param target_width: int, новая ширина изображения (по умолчанию 640)
    """
    image = cv2.imread(input_path)
    height, width, _ = image.shape
    target_height = int(height / width * target_width)
    resized_image = cv2.resize(image, (target_width, target_height), interpolation=cv2.INTER_AREA)
    cv2.imwrite(output_path, resized_image)

def resize_all_images(images_directory):
    """
    Изменение размера всех изображений в указанном каталоге.

    :param images_directory: str, путь к каталогу с изображениями
    """
    image_paths = glob.glob(os.path.join(images_directory, '*'))
    for img_path in tqdm(image_paths):
        resize_image(img_path, img_path)

In [ ]:
import numpy as np

def read_random_file_lines(file_names):
    """
    Открытие случайного файла из списка имен и чтение всех строк файла.

    :param file_names: list, список имен файлов
    :return: list, список прочитанных строк
    """
    
    random_index = np.random.randint(0, len(file_names))

    with open(file_names[random_index]) as file:
        lines = file.readlines()

    return lines


file_lines = read_random_file_lines(names)
print(file_lines)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def display_random_image_with_bounding_boxes(file_names):
    """
    Отображает случайное изображение с ограничивающими рамками.

    :param file_names: list, список имен файлов
    """
    # Выбор случайного индекса из списка имен файлов
    random_index = np.random.randint(0, len(file_names))

    # Чтение данных ограничивающих рамок из файла
    with open(file_names[random_index]) as file:
        lines = file.readlines()
        classes = [int(line[0]) for line in lines]
        objects = [line.rstrip()[2:].split() for line in lines]

    # Загрузка соответствующего изображения
    img_path = file_names[random_index].replace('txt', 'jpg').replace('labels', 'images')
    img = cv2.imread(img_path)

    # Рисование ограничивающих рамок на изображении
    for class_, obj in zip(classes, objects):
        x, y, w, h = [float(coord) for coord in obj]
        img_h, img_w = img.shape[:2]
        x = int(x * img_w)
        w = int(w * img_w)
        y = int(y * img_h)
        h = int(h * img_h)
        color = (255, 100, 50)
        cv2.rectangle(img, (int(x - w/2), int(y - h/2)), (int(x + w/2), int(y + h/2)), color, 4)

    # Отображение изображения
    plt.figure(figsize=(8, 8))
    plt.imshow(img[:, :, ::-1])
    plt.axis('off')
    plt.show()

    print(f'number of bounding boxes : {len(classes)}')
    print(f'Shape of the image : {img.shape}')

# Пример использования:
display_random_image_with_bounding_boxes(names)

In [ ]:
os.makedirs('Yolo/images', exist_ok= True)
os.makedirs('Yolo/labels', exist_ok= True)
os.makedirs('Yolo/images/train', exist_ok= True)
os.makedirs('Yolo/images/val', exist_ok= True)
os.makedirs('Yolo/labels/train', exist_ok= True)
os.makedirs('Yolo/labels/val', exist_ok= True)

In [ ]:
import os
from shutil import copy
from tqdm import tqdm

def split_train_val_data(src_labels_dir, src_images_dir, dest_labels_dir, dest_images_dir, val_ratio=0.85, random_seed=101):
    np.random.seed(random_seed)

    for subfolder in ['train', 'val']:
        src_image_subfolder = os.path.join(src_images_dir, subfolder)
        src_label_subfolder = os.path.join(src_labels_dir, subfolder)
        dest_image_subfolder = os.path.join(dest_images_dir, subfolder)
        dest_label_subfolder = os.path.join(dest_labels_dir, subfolder)

        image_names = os.listdir(src_image_subfolder)
        random_vector = np.random.rand(len(image_names))

        for i, img_name in tqdm(enumerate(image_names)):
            random_value = random_vector[i]

            if random_value > val_ratio:  # Validation
                copy(os.path.join(src_image_subfolder, img_name), os.path.join(dest_image_subfolder, img_name))
                copy(os.path.join(src_label_subfolder, img_name.split('.')[0] + '.txt'), os.path.join(dest_label_subfolder, img_name.split('.')[0] + '.txt'))
            else:  # Train
                copy(os.path.join(src_image_subfolder, img_name), os.path.join(dest_image_subfolder, img_name))
                copy(os.path.join(src_label_subfolder, img_name.split('.')[0] + '.txt'), os.path.join(dest_label_subfolder, img_name.split('.')[0] + '.txt'))

src_labels_address = 'newDataset/labels/'
src_imgs_address = 'newDataset/images/'
dest_labels_address = 'Yolo/labels/'
dest_imgs_address = 'Yolo/images/'

split_train_val_data(src_labels_address, src_imgs_address, dest_labels_address, dest_imgs_address)

In [ ]:
names = g('Yolo/images/train/*')
print(f'There are {len(names)}  images')

In [ ]:
import os
from IPython.display import clear_output

def setup_yolov5_repo(repo_url, requirements_path, yaml_path, train_dir, val_dir, class_count, class_names):
    """
    Клонирует репозиторий YOLOv5, устанавливает требуемые зависимости и создает dataset.yaml файл.

    :param repo_url: str, URL репозитория YOLOv5
    :param requirements_path: str, путь к файлу с требованиями
    :param yaml_path: str, путь к файлу dataset.yaml, который будет создан
    :param train_dir: str, путь к каталогу с тренировочными изображениями
    :param val_dir: str, путь к каталогу с валидационными изображениями
    :param class_count: int, количество классов в наборе данных
    :param class_names: list, список имен классов
    """

    # Клонирование репозитория YOLOv5 и установка требуемых зависимостей
    !git clone {repo_url}
    !pip install -qr {requirements_path}
    %cd yolov5
    clear_output()

    # Создание dataset.yaml файла
    with open(yaml_path, 'w') as yaml_file:
        yaml_file.write(f"train: {train_dir}\n")
        yaml_file.write(f"val: {val_dir}\n")
        yaml_file.write(f"nc: {class_count}\n")
        yaml_file.write(f"names: {class_names}\n")

# Пример использования функции:
repo_url = "https://github.com/ultralytics/yolov5.git"
requirements_path = "yolov5/requirements.txt"
yaml_path = "data/dataset.yaml"
train_dir = "newDataset/images/train"
val_dir = "newDataset/images/val"
class_count = 1
class_names = ["Face"]

setup_yolov5_repo(repo_url, requirements_path, yaml_path, train_dir, val_dir, class_count, class_names)

In [ ]:
import requests
def download_file(url, filename):
    response = requests.get(url, stream=True)
    with open(filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)

yolov5s_url = 'https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5x6.pt'
yolov5s_filename = 'yolov5x6.pt'

print(f'Downloading {yolov5s_filename}...')
download_file(yolov5s_url, yolov5s_filename)
print(f'{yolov5s_filename} is ready.')

In [ ]:
!python train.py --img 1280 --batch 3 --workers 2 --epochs 50\
  --weights yolov5x6.pt\
  --data data/dataset.yaml